## Setting up Your Python Environment
per: https://blog.roboflow.com/how-to-use-segment-anything-model-sam/

In [19]:
!pip install \
'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q roboflow supervision
!wget -q \
'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'
!pip install torch torchvision

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-y9jfwmg2
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-y9jfwmg2
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [24]:
import geopandas as gpd
import numpy as np
import rasterio
from PIL import Image
from shapely.geometry import Polygon
import torch
from segment_anything import sam_model_registry

## Loading the Segment Anything Model

In [25]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # use cpu if no access to gpu
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint="/home/jovyan/swe_forecasting_prod/notebooks/sam_vit_h_4b8939.pth")
sam.to(device=DEVICE)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [39]:
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image

In [42]:
# Function to convert GeoTIFF to JPEG
def convert_geotiff_to_jpeg(geotiff_path, jpeg_path):
    with rasterio.open(geotiff_path) as src:
        # Read the image data
        band_data = src.read(1)  # Assuming single band image for SWE
        # img_array = np.interp(img_array, (img_array.min(), img_array.max()), (0, 255)).astype(np.uint8)

        # Step 2: Normalize the data to 0-1 for colormap application
        band_min, band_max = band_data.min(), band_data.max()
        normalized_band = (band_data - band_min) / (band_max - band_min)
        
        # Step 3: Apply a colormap
        colormap = cm.get_cmap('viridis')  # Choose a colormap (e.g., 'viridis', 'jet', 'plasma')
        colored_image = colormap(normalized_band)  # Apply colormap
        colored_image = (colored_image[:, :, :3] * 255).astype(np.uint8)  # Convert to RGB and scale to 0-255
        
        # Step 4: Save the RGB image as JPEG
        jpeg_image = Image.fromarray(colored_image)
        jpeg_image.save(jpeg_path, 'JPEG')
        
        print("Single-band GeoTIFF has been successfully converted to an RGB JPEG image.")


        # # Convert to PIL Image
        # img = Image.fromarray(img_array)
        # img = img.convert('L')  # Convert to grayscale if needed

        # # Save as JPEG
        # img.save(jpeg_path)
        
# Paths to input and output files
geotiff_path = '/home/jovyan/shared-public/ml_swe_monitoring_prod/swe_predicted_2023-12-15_median_filtered.tif'
jpeg_path = '/home/jovyan/shared-public/ml_swe_monitoring_prod/swe_predicted_2023-12-15_median_filtered.jpg'
shapefile_path = '/home/jovyan/shared-public/ml_swe_monitoring_prod/swe_predicted_2023-12-15_median_filtered.shp'
    
# Convert GeoTIFF to JPEG
convert_geotiff_to_jpeg(geotiff_path, jpeg_path)


Single-band GeoTIFF has been successfully converted to an RGB JPEG image.


## Automated Mask (Instance Segmentation) Generation with SAM

In [43]:
import cv2
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)

image_bgr = cv2.imread(jpeg_path) # this is the jpeg of our swe prediction 

tile_size = (666, 666)  # Define the size of the tile
top_left_x, top_left_y = 0, 0  # Define the position of the top-left corner of the tile

# Calculate the bottom-right corner of the tile
bottom_right_x = top_left_x + tile_size[0]
bottom_right_y = top_left_y + tile_size[1]

# Extract the tile from the image
tile = image_bgr[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

image_rgb = cv2.cvtColor(tile, cv2.COLOR_BGR2RGB)
result = mask_generator.generate(image_rgb)

In [46]:
import supervision as sv

mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections.from_sam(sam_result=result)

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[image_bgr, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)
     

IndexError: boolean index did not match indexed array along dimension 1; dimension is 694 but corresponding boolean dimension is 666